https://mp.weixin.qq.com/s/GEgWHc8JngNsZYHUtBjHNg

https://discussion.datafountain.cn/questions/2413?shareFromPage=PAGE_QUESTION

In [1]:
import numpy as np
import pandas as pd

In [2]:
train_sales  = pd.read_csv('data_origin/train/train_sales_data.csv')

evaluation_public = pd.read_csv('data_origin/evaluation_public.csv')

In [23]:
def rmse(predict,real,xishu,month):
    rmse1 = np.sqrt((((predict - (real / month) ).map(lambda x:x**2))*xishu)).mean()
    return rmse1

In [4]:
model_name = (evaluation_public[['adcode','model']].drop_duplicates()).reset_index()

In [5]:
model_name.head()

,index,adcode,model
0,0,310000,3c974920a76ac9c1
1,1,530000,3c974920a76ac9c1
2,2,150000,3c974920a76ac9c1
3,3,110000,3c974920a76ac9c1
4,4,510000,3c974920a76ac9c1


In [6]:
train_sales.head(2)

,province,adcode,model,bodyType,regYear,regMonth,salesVolume
0,上海,310000,3c974920a76ac9c1,SUV,2016,1,292
1,云南,530000,3c974920a76ac9c1,SUV,2016,1,466


In [7]:
np.arange(5-1,-1,-1)

array([4, 3, 2, 1, 0])

In [14]:
np.arange(len(train_sales)-1,-1,-1)

array([31679, 31678, 31677, ...,     2,     1,     0])

In [24]:
all_result = None
for i in np.arange(0,len(model_name)):
    # 把同省份同车型的取出来
    adcode_mid = model_name[model_name['index'] == i]['adcode'][i]
    model_mid = model_name[model_name['index'] == i]['model'][i]
    train_sales_mid = train_sales[(train_sales['adcode']==adcode_mid)&(train_sales['model']==model_mid)]
    train_sales_mid = train_sales_mid.sort_values(['regYear','regMonth'])
#     print(train_sales_mid)
#     break
    # 添加衰减系数 
    train_sales_mid['xishu'] = np.arange(len(train_sales_mid)-1,-1,-1)
#     train_sales_mid['xishu'] = 0
#     train_sales_mid.loc[train_sales_mid.regYear==2017, 'xishu'] = 12
#     train_sales_mid['xishu'] = 24 - (train_sales_mid['xishu'] + train_sales_mid['regMonth'])
#     print(train_sales_mid['xishu'])
#     break
    train_sales_mid['xishu'] = train_sales_mid['xishu'].map(lambda x:1/(x+1))#map(lambda x:np.exp(-0.1*x))
    
    # 每个月销量均值，进行最大值归一化
    train_sales_month = train_sales_mid.groupby('regMonth',as_index=False)['salesVolume'].agg({'mean_sum':'mean'})
    train_sales_month['month'] = train_sales_month['mean_sum'] / train_sales_month['mean_sum'].max()
    train_sales_mid = pd.merge(train_sales_mid,train_sales_month[['regMonth','month']],on=['regMonth'],how='left')

    train_sales_mid_max = train_sales_mid['salesVolume'].max()
    train_sales_mid_min = train_sales_mid['salesVolume'].min()
    evaluation_public_mid = evaluation_public[(evaluation_public['adcode']==adcode_mid)&(evaluation_public['model']==model_mid)]
    evaluation_public_mid = pd.merge(evaluation_public_mid,train_sales_month[['regMonth','month']],on=['regMonth'],how='left')
    
    
    rmse_min = np.inf
    best_num = None
    for num in np.arange(train_sales_mid_min+0.3 ,train_sales_mid_max,0.3):
        rmse1 = rmse(num,train_sales_mid['salesVolume'],train_sales_mid['xishu'],train_sales_mid['month'])
        if rmse1 < rmse_min:
            rmse_min = rmse1
            best_num = num
    print(best_num)
    evaluation_public_mid['forecastVolum'] = best_num 
    evaluation_public_mid['forecastVolum'] = evaluation_public_mid['forecastVolum']*evaluation_public_mid['month']
    all_result = pd.concat([all_result,evaluation_public_mid],axis=0)

341.60000000000787
440.90000000000936
212.39999999999915
413.8000000000096
531.7000000000105
214.89999999999867
532.1000000000106
249.89999999999884
3328.0999999996366
436.40000000000805
889.8000000000153
238.49999999999847
348.900000000008
468.3000000000092
864.9000000000199
582.0000000000134
431.2000000000104
366.9000000000082
877.9000000000169
162.29999999999916
364.09999999999764
407.50000000000983
582.4000000000121
554.1000000000126
359.2999999999978
542.9000000000126
1094.5000000000236
694.30000000002
1539.7000000000414
571.4000000000154
11853.50000000562
1245.6000000000315
2840.8999999997704
668.900000000019
1230.9000000000333
1272.9000000000333
2558.3999999997677
1136.1000000000288
987.6000000000222
1908.6999999997981
2514.5999999997375
391.3000000000099
572.1000000000165
1179.9000000000337
204.1999999999988
376.10000000000645
286.29999999999814
630.8000000000142
470.80000000001075
163.89999999999907
420.20000000001016
197.39999999999873
1949.9999999997876
210.99999999999903
47

1032.6000000000292
2125.7000000000685
551.9000000000149
4158.099999999594
1016.0000000000298
4236.899999999557
3752.2999999995636
3213.399999999633
1298.1000000000356
1311.9000000000397
1015.0000000000277
1390.500000000037
634.7000000000162
1184.0000000000357
1112.8000000000256
1134.7000000000307
428.00000000000887
372.0000000000084
1031.1000000000229
1030.000000000022
932.4000000000217
2490.199999999753
379.6000000000091
1111.800000000029
312.0999999999981
2904.99999999972
478.10000000001145
2028.9999999998067
1677.4999999998333
2550.499999999737
561.6000000000107
637.3000000000145
501.6000000000108
428.2000000000098
195.89999999999878
595.7000000000155
523.900000000014
117.50000000000021
108.10000000000018
167.09999999999906
102.00000000000017
376.8000000000094
859.5000000000223
1073.4000000000262
448.10000000001156
166.9999999999991
126.79999999999913
415.8000000000094
180.8999999999986
1071.2000000000257
717.0000000000205
133.59999999999926
341.7999999999975
350.8999999999978
84.10

267.39999999999856
191.99999999999918
435.20000000001016
62.90000000000009
46.300000000000054
92.99999999999946
154.59999999999937
28.300000000000043
40.20000000000007
70.09999999999964
337.4000000000075
511.3000000000117
196.49999999999898
264.09999999999843
671.4000000000163
468.7000000000116
969.8000000000254
351.0000000000083
4458.19999999952
467.20000000001085
1040.5000000000202
460.00000000001046
785.0000000000191
700.9000000000171
1283.09999999989
904.2000000000239
737.6000000000201
851.8000000000218
563.800000000014
242.9999999999987
284.3999999999979
376.4999999999975
469.09999999999667
168.8999999999989
251.79999999999848
738.9000000000174
485.00000000001023
473.40000000001214
1122.7000000000285
541.0000000000134
3251.499999999619
169.89999999999912
1233.8999999998966
322.1999999999977
1051.0000000000246
795.0000000000202
1698.8999999998257
748.9000000000187
759.7000000000198
511.4000000000125
1185.400000000028
196.99999999999912
295.99999999999807
658.3000000000168
315.69999

In [25]:
all_result = all_result.sort_values(['id'])   

In [26]:
dfr = all_result[['id', 'forecastVolum']].round().astype(int)

In [28]:
dfr.head()

,id,forecastVolum
0,1,297
0,2,441
0,3,212
0,4,414
0,5,532


In [11]:
dfr.head()

,id,forecastVolum
0,1,287
0,2,416
0,3,205
0,4,380
0,5,500


In [29]:

dfr.to_csv('/Users/luoyonggui/Downloads/ds_ccf_car_sales.csv', index=False)

In [13]:
dfr.forecastVolum.mean()

500.7301136363636

In [14]:
dfr.shape

(5280, 2)